# Writing C for Python

## CPython API and C
Since Python itself is built from C, it natively comes installed with a C code library containing all of Python's C code, the library contains all of Python's base logic like integer behavior, object definitions, types, ...ect. In that library there is a header file labeled `Python.h` which is the entry point of the Python C API. This enables programmers to directly interface with Python's native C environment and run their own C code from a Python module. To get started all you need is Python already installed on your machine and a copy of the `include` folder where python is installed `C:\Users\user\AppData\Local\Programs\Python\Python311\` by default on Windows and `/usr/local/bin/python3` on Linux. Once a new C source file is created you can import `#include <Python.h>` directly and start using the API in the C source file. There is a lot of functionality CPython offers which is all compiled into a [documentation by Python](https://docs.python.org/3/c-api/index.html).

## What considerations should I make when trying to integrate my C code with CPython?
Since CPython interfaces with python's native C environet, any data that is processed by Python will be its own data type. This data type from the pure C side looks like a pointer to a struct as `PyObject* var` but could represent anything in the Python native types from list, int, float, str, dict, ect. Since this is a pointer to a struct in pure C terms CPython has built-in functions to convert their structures to native C types to perform native calculations using longs, floats, doubles, int, char, ect. For example to convert a Python float to a C double you can use `PyFloat_AsDouble(PyObject *Float)` or to convert a Python int to a C long use `PyLong_AsLong(PyObject *int)`. From here you can start writing C code and treating data types normally. There is another consideration in how methods are declared and used. Since this code will eventually be called by Python, the return will need to be type `PyObject*` alongside any arguments. A method declaration generally looks like:

`static PyObject* method(PyObject *self, PyObject *args){}`

If you want to pass arguments, those are loaded intermediately by the Python environment into a `PyObject` and accessed through `PyArg_ParseTuple(args,"var_type_string",&var)` and for multiple args `PyArg_ParseTuple(args,"Var_type_1Vr_type_2Var_type_n",&var_1,&var_2,&var_n)`.

## How is this ran from Python
This process is a bit different from compiling C code normally, instead of using a compiler like GCC Python requires a script using their native `setuptools` library which defines functions and objects used to compile the CPython-based code into a Python extended module. A basic script setup usually looks like:


```
from setuptools import setup, Extension

module = Extension("ModuleName",sources=['ModuleName_Source.c'])
setup(name="ModuleName",version='1.0',description="desc",ext_modules=[module])
```

If the code is written correctly then Python will create a `.ext` file in a subdirectory labeled `build` and under another subdirectory labeled as your operating system, computers architecture, and CPython version, for example : `lib.win-amd64-cpython-311`. You can copy and paste this file anywhere you have a Python file ready to import. Once the file is in the same location you can import the CPython code as any other Python library:

`import ModuleName`

## How should modules be created in C code?
Python has 3 separate declarations that need to be made; PyMethodDef, PyModuleDef, PyInit_ModuleName each being a struct of methods defined in your code, a struct of module information, and a initialization function which is usually defined as the following:

```
static PyMethodDef Functions[] = {
	{"MethodName",MethodName,METH_VARARGS,""},
	{NULL,NULL,0,NULL}
};

static struct PyModuleDef CStats = {
	PyModuleDef_HEAD_INIT,
	"ModuleName",
	NULL,
	-1,
	Functions
};

PyMODINIT_FUNC PyInit_ModuleName(void){
	return PyModule_Create(&ModuleName);
}
```

## When to use CPython over Python and conclusions
Python is a pretty decent language in terms of speed, there is usually an exact reason computationally why someone would want to integrate C code with their Python project as a ext module such as processing large datasets where low-level control could benefit, more "pure" computations where you want only a specific operation being performed instead of some other operations that are ultimately useless, the usual reasons someone would choose C over any other programming language to perform an operation. For this project my reasons are the same, since some of the datasets that could be theoretically downloaded could be millions of data points. If we were to assume minute intervals from January 1st 2000 to January 1st 2020 we would have 5,040 trading days total (20 * 252), 35,280 hours ((9AM-4PM) 7 * 252), or 2,116,800 minutes/data points (60*5,280) for one variable, since there are 7 this is scaled up to 14,817,600 data points for 20 years worth of minute intervals. This can get very slow computationally and given lower-level algorithms this can become a much less intensive task and therefore a shorter wait for the front-end.